In [ ]:
# default_exp neuralnets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# neuralnets

> Implementations of RL policies, value functions, and actor-critic networks.

In [ ]:
#hide
from nbdev import *

In [ ]:
%nbdev_export
import numpy as np
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import signal
import gym
from scipy.signal import lfilter
from typing import Optional, Iterable, List, Dict, Callable, Union, Tuple
from rl_bolts.env_wrappers import ToTorchWrapper

In [ ]:
%nbdev_export
class MLP(nn.Module):
    r"""
    A class for building a simple MLP network.

    Args:
    - layer_sizes (list or tuple): Layer sizes for the network.
    - activations (Function): Activation function for MLP net.
    - out_act (Function): Output activation function
    - out_squeeze (bool): Whether to squeeze the output of the network.
    """

    def __init__(
        self,
        layer_sizes: Union[List, Tuple],
        activations: Optional[Callable] = torch.tanh,
        out_act: Optional[bool] = None,
        out_squeeze: Optional[bool] = False,
    ):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.activations = activations
        self.out_act = out_act
        self.out_squeeze = out_squeeze

        for i, l in enumerate(layer_sizes[1:]):
            self.layers.append(nn.Linear(layer_sizes[i], l))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass for MLP network"""
        for l in self.layers[:-1]:
            x = self.activations(l(x))

        if self.out_act is None:
            x = self.layers[-1](x)
        else:
            x = self.out_act(self.layers[-1](x))

        return torch.squeeze(x, -1) if self.out_squeeze else x

In [ ]:
#hide
mlp = MLP((3, 5, 7, 1))
inp = torch.randn(3)
assert mlp(inp) is not None, "MLP is failing to forward propagate."
assert mlp(inp).shape[0] == 1, "MLP is mapping to wrong dimension."

In [ ]:
show_doc(MLP)

<h2 id="MLP" class="doc_header"><code>class</code> <code>MLP</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>MLP</code>(**`layer_sizes`**:`Union`\[`List`\[`T`\], `Tuple`\], **`activations`**:`Optional`\[`Callable`\]=*`'tanh'`*, **`out_act`**:`Optional`\[`bool`\]=*`None`*, **`out_squeeze`**:`Optional`\[`bool`\]=*`False`*) :: `Module`

A class for building a simple MLP network.

Args:
- layer_sizes (list or tuple): Layer sizes for the network.
- activations (Function): Activation function for MLP net.
- out_act (Function): Output activation function
- out_squeeze (bool): Whether to squeeze the output of the network.

In [ ]:
%nbdev_export
class CNN(nn.Module):
    """
    Create a PyTorch CNN module.
    
    Args:
    - input_channels (int): number of channels in the input
    - input_height (int): size of one side of input (currently assumes square input)
    - output_size (int): size of network output
    - kernel_size (int): Convolutional kernel size
    - stride (int): convolutional kernel stride
    - channels (list or tuple): List of channel sizes for each convolutional layer
    - linear_layer_sizes (list or tuple): list of (if any) sizes of linear layers to add after convolutional layers
    - activation (callable): activation function
    - output_activation (int): if any, activation to apply to the output layer
    - dropout_layers (list or tuple): if any, layers to apply dropout to
    - dropout_p (float): probability of dropout to use
    - out_squeeze (bool): whether to squeeze the output
    """
    def __init__(self, 
            input_channels: int,
            input_height: int,
            output_size: int,
            kernel_size: Optional[int] = 3,
            stride: Optional[int] = 1,
            channels: Optional[list] = [64, 64],
            linear_layer_sizes: Optional[list] = [512],
            activation: Optional[Callable] = torch.relu,
            output_activation: Optional[Callable] = None,
            dropout_layers: Optional[list] = None,
            dropout_p: Optional[float] = None,
            out_squeeze: Optional[bool] = False
        ):
        
        super(CNN, self).__init__()

        conv_sizes = [input_channels] + channels
        self.layers = nn.ModuleList()
        self.activation = activation
        self.output_activation = output_activation
        self.out_squeeze = out_squeeze

        self.dropout_p = dropout_p
        self.dropout_layers = dropout_layers

        self.hw=input_height
        for i, l in enumerate(conv_sizes[1:]):
            self.hw = conv2d_output_size(kernel_size=kernel_size, stride=stride, sidesize=self.hw)
            self.layers.append(nn.Conv2d(conv_sizes[i], l, kernel_size=kernel_size, stride=stride))

        self.hw = (self.hw, self.hw)
        conv_out_size = 1
        for num in self.hw:
            conv_out_size *= num
        conv_out_size *= conv_sizes[-1]
        linear_sizes = [conv_out_size] + linear_layer_sizes + [output_size]
        self.layers.append(nn.Flatten())
        for i, l in enumerate(linear_sizes[1:]):
            self.layers.append(nn.Linear(linear_sizes[i], l))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for l in self.layers[:-1]:
            x = self.activation(l(x))

            if self.dropout_layers is not None and l in self.dropout_layers:
                x = F.dropout(x, p=self.dropout_p)

        if self.output_activation is None:
            x = self.layers[-1](x)
        else:
            x = self.output_activation(self.layers[-1](x))

        return x.squeeze() if self.out_squeeze else x

In [ ]:
show_doc(CNN)

<h2 id="CNN" class="doc_header"><code>class</code> <code>CNN</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CNN</code>(**`input_channels`**:`int`, **`input_height`**:`int`, **`output_size`**:`int`, **`kernel_size`**:`Optional`\[`int`\]=*`3`*, **`stride`**:`Optional`\[`int`\]=*`1`*, **`channels`**:`Optional`\[`list`\]=*`[64, 64]`*, **`linear_layer_sizes`**:`Optional`\[`list`\]=*`[512]`*, **`activation`**:`Optional`\[`Callable`\]=*`'relu'`*, **`output_activation`**:`Optional`\[`Callable`\]=*`None`*, **`dropout_layers`**:`Optional`\[`list`\]=*`None`*, **`dropout_p`**:`Optional`\[`float`\]=*`None`*, **`out_squeeze`**:`Optional`\[`bool`\]=*`False`*) :: `Module`

Create a PyTorch CNN module.

Args:
- input_channels (int): number of channels in the input
- input_height (int): size of one side of input (currently assumes square input)
- output_size (int): size of network output
- kernel_size (int): Convolutional kernel size
- stride (int): convolutional kernel stride
- channels (list or tuple): List of channel sizes for each convolutional layer
- linear_layer_sizes (list or tuple): list of (if any) sizes of linear layers to add after convolutional layers
- activation (callable): activation function
- output_activation (int): if any, activation to apply to the output layer
- dropout_layers (list or tuple): if any, layers to apply dropout to
- dropout_p (float): probability of dropout to use
- out_squeeze (bool): whether to squeeze the output

In [ ]:
%nbdev_export
class Actor(nn.Module):
    """
    Barebones class structure for an Actor.
    """
    def action_distribution(self, states):
        raise NotImplementedError

    def logprob_from_distribution(self, policy, action):
        raise NotImplementedError

    def forward(self, x, a = None):
        """
        Forward pass for an policy.

        Args:
        - x (torch.Tensor): Input state from the environment.
        - a (torch.Tensor): Action that was taken.

        Returns:
        - policy (PyTorch distribution): The policy distribution.
        - logp_a (torch.Tensor): Log-probability of input action under the policy distribution.
        """
        policy = self.action_distribution(x)
        logp_a = None
        if a is not None:
            logp_a = self.logprob_from_distribution(policy, a)
        return policy, logp_a

In [ ]:
show_doc(Actor)

<h2 id="Actor" class="doc_header"><code>class</code> <code>Actor</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Actor</code>() :: `Module`

Barebones class structure for an Actor.

In [ ]:
show_doc(Actor.forward)

<h4 id="Actor.forward" class="doc_header"><code>Actor.forward</code><a href="__main__.py#L12" class="source_link" style="float:right">[source]</a></h4>

> <code>Actor.forward</code>(**`x`**, **`a`**=*`None`*)

Forward pass for an policy.

Args:
- x (torch.Tensor): Input state from the environment.
- a (torch.Tensor): Action that was taken.

Returns:
- policy (PyTorch distribution): The policy distribution.
- logp_a (torch.Tensor): Log-probability of input action under the policy distribution.

In [ ]:
%nbdev_export
class CategoricalPolicy(Actor):
    r"""
    A class for a Categorical Policy network. Used in discrete action space environments.

    The policy is an `MLP`.

    Args:
    - state_features (int): Dimensionality of the state space.
    - action_dim (int): Dimensionality of the action space.
    - hidden_sizes (list or tuple): Hidden layer sizes.
    - activation (Function): Activation function for the network.
    - out_activation (Function): Output activation function for the network.
    """

    def __init__(
        self,
        state_features: int,
        action_dim: int,
        hidden_sizes: Union[List, Tuple],
        activation: Callable,
        out_activation: Callable,
    ):
        super().__init__()
        self.net = MLP(
            [state_features] + list(hidden_sizes) + [action_dim], activations=activation
        )

    def action_distribution(self, x: torch.Tensor):
        """
        Defines action distribution conditioned on input state.

        Args:
        - x(torch.Tensor): input state

        Returns:
        - Categorical distribution: Policy over the action space.
        """
        logits = self.net(x)
        return torch.distributions.Categorical(logits=logits)

    def logprob_from_distribution(self, policy: torch.distributions.Distribution, actions: torch.Tensor):
        """
        Calculate the log-probability of an action under a policy.

        Args:
        - policy (torch.distributions.Distribution): The policy distribution over input state.
        - actions (torch.Tensor): Actions to take log probability of.

        Returns:
        - log_probs (torch.Tensor): Log-probabilities of actions under the policy distribution.
        """
        return policy.log_prob(actions)

In [ ]:
show_doc(CategoricalPolicy)

<h2 id="CategoricalPolicy" class="doc_header"><code>class</code> <code>CategoricalPolicy</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CategoricalPolicy</code>(**`state_features`**:`int`, **`action_dim`**:`int`, **`hidden_sizes`**:`Union`\[`List`\[`T`\], `Tuple`\], **`activation`**:`Callable`, **`out_activation`**:`Callable`) :: [`Actor`](/rl_bolts/neuralnets#Actor)

A class for a Categorical Policy network. Used in discrete action space environments.

The policy is an [`MLP`](/rl_bolts/neuralnets#MLP).

Args:
- state_features (int): Dimensionality of the state space.
- action_dim (int): Dimensionality of the action space.
- hidden_sizes (list or tuple): Hidden layer sizes.
- activation (Function): Activation function for the network.
- out_activation (Function): Output activation function for the network.

In [ ]:
show_doc(CategoricalPolicy.logprob_from_distribution)

<h4 id="CategoricalPolicy.logprob_from_distribution" class="doc_header"><code>CategoricalPolicy.logprob_from_distribution</code><a href="__main__.py#L42" class="source_link" style="float:right">[source]</a></h4>

> <code>CategoricalPolicy.logprob_from_distribution</code>(**`policy`**:`Distribution`, **`actions`**:`Tensor`)

Calculate the log-probability of an action under a policy.

Args:
- policy (torch.distributions.Distribution): The policy distribution over input state.
- actions (torch.Tensor): Actions to take log probability of.

Returns:
- log_probs (torch.Tensor): Log-probabilities of actions under the policy distribution.

In [ ]:
show_doc(CategoricalPolicy.action_distribution)

<h4 id="CategoricalPolicy.action_distribution" class="doc_header"><code>CategoricalPolicy.action_distribution</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>CategoricalPolicy.action_distribution</code>(**`x`**:`Tensor`)

Defines action distribution conditioned on input state.

Args:
- x(torch.Tensor): input state

Returns:
- Categorical distribution: Policy over the action space.

In [ ]:
%nbdev_export
class GaussianPolicy(Actor):
    r"""
    A class for a Gaussian Policy network. Used in continuous action space environments. The policy is an `MLP`.

    Args:
    - state_features (int): Dimensionality of the state space.
    - action_dim (int): Dimensionality of the action space.
    - hidden_sizes (list or tuple): Hidden layer sizes.
    - activation (Function): Activation function for the network.
    - out_activation (Function): Output activation function for the network.
    """

    def __init__(
        self,
        state_features: int,
        action_dim: int,
        hidden_sizes: Union[List, Tuple],
        activation: Callable,
        out_activation: Callable,
    ):
        super().__init__()

        self.net = MLP(
            [state_features] + list(hidden_sizes) + [action_dim],
            activations=activation,
            out_act=out_activation,
        )

        self.logstd = nn.Parameter(-0.5 * torch.ones(action_dim, dtype=torch.float32))

    def action_distribution(self, states):
        """
        Defines action distribution conditioned on input state.

        Args:
        - x(torch.Tensor): input state

        Returns:
        - Normal distribution: Policy over the action space.
        """
        mus = self.net(states)
        std = torch.exp(self.logstd)
        return torch.distributions.Normal(mus, std)

    def logprob_from_distribution(self, policy, actions):
        """
        Calculate the log-probability of an action under a policy.

        Args:
        - policy (torch.distributions.Distribution): The policy distribution over input state.
        - actions (torch.Tensor): Actions to take log probability of.

        Returns:
        - log_probs (torch.Tensor): Log-probabilities of actions under the policy distribution.
        """
        return policy.log_prob(actions).sum(axis=-1)

In [ ]:
show_doc(GaussianPolicy)

<h2 id="GaussianPolicy" class="doc_header"><code>class</code> <code>GaussianPolicy</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>GaussianPolicy</code>(**`state_features`**:`int`, **`action_dim`**:`int`, **`hidden_sizes`**:`Union`\[`List`\[`T`\], `Tuple`\], **`activation`**:`Callable`, **`out_activation`**:`Callable`) :: [`Actor`](/rl_bolts/neuralnets#Actor)

A class for a Gaussian Policy network. Used in continuous action space environments. The policy is an [`MLP`](/rl_bolts/neuralnets#MLP).

Args:
- state_features (int): Dimensionality of the state space.
- action_dim (int): Dimensionality of the action space.
- hidden_sizes (list or tuple): Hidden layer sizes.
- activation (Function): Activation function for the network.
- out_activation (Function): Output activation function for the network.

In [ ]:
show_doc(GaussianPolicy.action_distribution)

<h4 id="GaussianPolicy.action_distribution" class="doc_header"><code>GaussianPolicy.action_distribution</code><a href="__main__.py#L32" class="source_link" style="float:right">[source]</a></h4>

> <code>GaussianPolicy.action_distribution</code>(**`states`**)

Defines action distribution conditioned on input state.

Args:
- x(torch.Tensor): input state

Returns:
- Normal distribution: Policy over the action space.

In [ ]:
show_doc(GaussianPolicy.logprob_from_distribution)

<h4 id="GaussianPolicy.logprob_from_distribution" class="doc_header"><code>GaussianPolicy.logprob_from_distribution</code><a href="__main__.py#L46" class="source_link" style="float:right">[source]</a></h4>

> <code>GaussianPolicy.logprob_from_distribution</code>(**`policy`**, **`actions`**)

Calculate the log-probability of an action under a policy.

Args:
- policy (torch.distributions.Distribution): The policy distribution over input state.
- actions (torch.Tensor): Actions to take log probability of.

Returns:
- log_probs (torch.Tensor): Log-probabilities of actions under the policy distribution.

In [ ]:
%nbdev_export
class ActorCritic(nn.Module):
    r"""
    An Actor Critic class for Policy Gradient algorithms.

    Has built-in capability to work with continuous (gym.spaces.Box) and discrete (gym.spaces.Discrete) action spaces.
    The policy and value function are both `MLP`.

    If working with a different action space,
    the user can pass in a custom policy class for that action space as an argument.

    Args:
    - state_features (int): Dimensionality of the state space.
    - action_space (gym.spaces.Space): Action space of the environment.
    - hidden_sizes (list or tuple): Hidden layer sizes.
    - activation (Function): Activation function for the network.
    - out_activation (Function): Output activation function for the network.
    - policy (nn.Module): Custom policy class for an environment where the action space is not gym.spaces.Box or gym.spaces.Discrete

    """

    def __init__(
        self,
        state_features: int,
        action_space: int,
        hidden_sizes: Optional[Union[Tuple, List]] = (32, 32),
        activation: Optional[Callable] = torch.tanh,
        out_activation: Optional[Callable] = None,
        policy: Optional[nn.Module] = None,
    ):
        super(ActorCritic, self).__init__()

        obs_dim = state_features

        if isinstance(action_space, gym.spaces.Discrete):
            act_dim = action_space.n
            pol = CategoricalPolicy

        elif isinstance(action_space, gym.spaces.Box):
            act_dim = action_space.shape[0]
            pol = GaussianPolicy
        else:
            act_dim = action_space
            pol = policy

        self.policy = pol(
            obs_dim,
            act_dim,
            hidden_sizes,
            activation,
            out_activation
        )

        self.value_f = MLP(
            [state_features] + list(hidden_sizes) + [1],
            activations=activation,
            out_squeeze=True,
        )

    def step(self, x: torch.Tensor):
        """
        Get action, action log probability, and value estimate for an input state.

        Args:
        - x (torch.Tensor): input state.

        Returns:
        - action (torch.Tensor): Action chosen by the policy.
        - logp_action (torch.Tensor): Log probability of that action chosen by the policy.
        - value (torch.Tensor): Value estimate of the current state.
        """
        with torch.no_grad():
            policy = self.policy.action_distribution(x)
            action = policy.sample()
            logp_action = self.policy.logprob_from_distribution(policy, action)
            value = self.value_f(x)
        return action, logp_action, value

    def act(self, x: torch.Tensor):
        """
        Similar to `step`, but get only the action.

        Args:
        - x (torch.Tensor): input state

        Returns:
        - action (torch.Tensor): Action chosen by the policy.
        """
        return self.step(x)[0]

In [ ]:
show_doc(ActorCritic)

<h2 id="ActorCritic" class="doc_header"><code>class</code> <code>ActorCritic</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>ActorCritic</code>(**`state_features`**:`int`, **`action_space`**:`int`, **`hidden_sizes`**:`Union`\[`Tuple`, `List`\[`T`\], `NoneType`\]=*`(32, 32)`*, **`activation`**:`Optional`\[`Callable`\]=*`'tanh'`*, **`out_activation`**:`Optional`\[`Callable`\]=*`None`*, **`policy`**:`Optional`\[`Module`\]=*`None`*) :: `Module`

An Actor Critic class for Policy Gradient algorithms.

Has built-in capability to work with continuous (gym.spaces.Box) and discrete (gym.spaces.Discrete) action spaces.
The policy and value function are both [`MLP`](/rl_bolts/neuralnets#MLP).

If working with a different action space,
the user can pass in a custom policy class for that action space as an argument.

Args:
- state_features (int): Dimensionality of the state space.
- action_space (gym.spaces.Space): Action space of the environment.
- hidden_sizes (list or tuple): Hidden layer sizes.
- activation (Function): Activation function for the network.
- out_activation (Function): Output activation function for the network.
- policy (nn.Module): Custom policy class for an environment where the action space is not gym.spaces.Box or gym.spaces.Discrete

In [ ]:
show_doc(ActorCritic.step)

<h4 id="ActorCritic.step" class="doc_header"><code>ActorCritic.step</code><a href="__main__.py#L60" class="source_link" style="float:right">[source]</a></h4>

> <code>ActorCritic.step</code>(**`x`**:`Tensor`)

Get action, action log probability, and value estimate for an input state.

Args:
- x (torch.Tensor): input state.

Returns:
- action (torch.Tensor): Action chosen by the policy.
- logp_action (torch.Tensor): Log probability of that action chosen by the policy.
- value (torch.Tensor): Value estimate of the current state.

In [ ]:
show_doc(ActorCritic.act)

<h4 id="ActorCritic.act" class="doc_header"><code>ActorCritic.act</code><a href="__main__.py#L79" class="source_link" style="float:right">[source]</a></h4>

> <code>ActorCritic.act</code>(**`x`**:`Tensor`)

Similar to `step`, but get only the action.

Args:
- x (torch.Tensor): input state

Returns:
- action (torch.Tensor): Action chosen by the policy.

In [ ]:
%nbdev_export

class MLPQActor(nn.Module):
    r"""
    An actor for Q policy gradient algorithms. 
    
    The policy is an `MLP`. 
    The output from the policy network is scaled to action space limits on the forward pass.

    Args:
    - state_features (int): Dimensionality of the state space.
    - action_dim (int): Dimensionality of the action space.
    - hidden_sizes (list or tuple): Hidden layer sizes.
    - activation (Function): Activation function for the network.
    - action_limit (float or int): Limits of the action space.
    """

    def __init__(
        self,
        state_features: int,
        action_dim: int,
        hidden_sizes: Union[list, tuple],
        activation: Callable,
        action_limit: Union[float, int],
    ):
        super(MLPQActor, self).__init__()
        policy_layer_sizes = [state_features] + list(hidden_sizes) + [action_dim]
        self.policy = MLP(policy_layer_sizes, activation, torch.tanh)
        self.action_limit = action_limit

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Return output from the policy network scaled to the limits of the env action space.
        Args:
        - x (torch.Tensor): States from environment.
        
        Returns:
        - scaled_action (torch.Tensor): Action scaled to action space limits.
        """
        scaled_action = self.action_limit * self.policy(x)
        return scaled_action

In [ ]:
show_doc(MLPQActor)

<h2 id="MLPQActor" class="doc_header"><code>class</code> <code>MLPQActor</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>MLPQActor</code>(**`state_features`**:`int`, **`action_dim`**:`int`, **`hidden_sizes`**:`Union`\[`list`, `tuple`\], **`activation`**:`Callable`, **`action_limit`**:`Union`\[`float`, `int`\]) :: `Module`

An actor for Q policy gradient algorithms. 

The policy is an [`MLP`](/rl_bolts/neuralnets#MLP). 
The output from the policy network is scaled to action space limits on the forward pass.

Args:
- state_features (int): Dimensionality of the state space.
- action_dim (int): Dimensionality of the action space.
- hidden_sizes (list or tuple): Hidden layer sizes.
- activation (Function): Activation function for the network.
- action_limit (float or int): Limits of the action space.

In [ ]:
show_doc(MLPQActor.forward)

<h4 id="MLPQActor.forward" class="doc_header"><code>MLPQActor.forward</code><a href="__main__.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>MLPQActor.forward</code>(**`x`**:`Tensor`)

Return output from the policy network scaled to the limits of the env action space.
Args:
- x (torch.Tensor): States from environment.

Returns:
- scaled_action (torch.Tensor): Action scaled to action space limits.

In [ ]:
%nbdev_export
class MLPQFunction(nn.Module):
    r"""
    A Q function network for Q policy gradient methods. 

    The Q function is an `MLP`. It always takes in a (state, action) pair and returns a Q-value estimate for that pair.

    Args:
    - state_features (int): Dimensionality of the state space.
    - action_dim (int): Dimensionality of the action space.
    - hidden_sizes (list or tuple): Hidden layer sizes.
    - activation (Function): Activation function for the network.
    """

    def __init__(
        self,
        state_features: int,
        action_dim: int,
        hidden_sizes: Union[tuple, list],
        activation: Callable,
    ):
        super().__init__()
        self.qfunc = MLP(
            [state_features + action_dim] + list(hidden_sizes) + [1], activation
        )

    def forward(self, x: torch.Tensor, a: torch.Tensor) -> torch.Tensor:
        """
        Return Q-value estimate for state, action pair (x, a).
        
        Args:
        - x (torch.Tensor): Environment state.
        - a (torch.Tensor): Action taken by the policy.
        
        Returns:
        - q (torch.Tensor): Q-value estimate for state action pair.
        """
        q = self.qfunc(torch.cat([x, a], dim=-1))
        return torch.squeeze(q, -1)  # Critical to ensure q has right shape.


In [ ]:
show_doc(MLPQFunction)

<h2 id="MLPQFunction" class="doc_header"><code>class</code> <code>MLPQFunction</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>MLPQFunction</code>(**`state_features`**:`int`, **`action_dim`**:`int`, **`hidden_sizes`**:`Union`\[`tuple`, `list`\], **`activation`**:`Callable`) :: `Module`

A Q function network for Q policy gradient methods. 

The Q function is an [`MLP`](/rl_bolts/neuralnets#MLP). It always takes in a (state, action) pair and returns a Q-value estimate for that pair.

Args:
- state_features (int): Dimensionality of the state space.
- action_dim (int): Dimensionality of the action space.
- hidden_sizes (list or tuple): Hidden layer sizes.
- activation (Function): Activation function for the network.

In [ ]:
show_doc(MLPQFunction.forward)

<h4 id="MLPQFunction.forward" class="doc_header"><code>MLPQFunction.forward</code><a href="__main__.py#L27" class="source_link" style="float:right">[source]</a></h4>

> <code>MLPQFunction.forward</code>(**`x`**:`Tensor`, **`a`**:`Tensor`)

Return Q-value estimate for state, action pair (x, a).

Args:
- x (torch.Tensor): Environment state.
- a (torch.Tensor): Action taken by the policy.

Returns:
- q (torch.Tensor): Q-value estimate for state action pair.

In [ ]:
#hide
notebook2script()

Converted 00_utils.ipynb.
Converted 01_datasets.ipynb.
Converted 02_buffers.ipynb.
Converted 03_neuralnets.ipynb.
Converted 04_losses.ipynb.
Converted 05_env_wrappers.ipynb.
Converted 06_loops.ipynb.
Converted 07_algorithms.ipynb.
Converted index.ipynb.
